In [64]:
import requests
import pandas as pd

pd.set_option('display.max_rows', None)

Given a ticker, we will query the
- stock price
- available exercise prices
- available times to expiration
- risk-free interest rate
- volatility

and use black scholes formula to return the option price

Choosing Options:

- Choose Ticker
> see all expiries
- Choose Expiry
> See options chain for that Expiry
- Choose option by code
- Plug it into formula

or

- Choose Ticker, OptType, Expiry and Strike
- Plug into formula

In [88]:
API_KEY = "OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX"

MEMORY = {}

OPTIONS = {}

RENAME_MAP = {
    'CALL': {
        'strike': 'Strike', 
        'contractName': 'cName',
        'lastPrice': 'lastPrice', 
        'bid': 'cBidP', 
        'ask': 'cAskP',
        'volume': 'cVolume', 
        'impliedVolatility': 'impliedVolatility', 
        'delta': 'cDelta', 
        'gamma': 'Gamma', 
        'theta': 'Theta', 
        'vega': 'Vega', 
        'rho': 'Rho'
    },
    'PUT': {
        'strike': 'Strike', 
        'contractName': 'pName',
        'lastPrice': 'lastPrice', 
        'bid': 'pBidP', 
        'ask': 'pAskP',
        'volume': 'pVolume', 
        'impliedVolatility': 'impliedVolatility', 
        'delta': 'pDelta', 
        'gamma': 'Gamma', 
        'theta': 'Theta', 
        'vega': 'Vega', 
        'rho': 'Rho'
    }
}



In [96]:
def query_data(ticker):
    global OPTIONS
    global MEMORY
    global API_KEY
    global RENAME_MAP

    if ticker not in OPTIONS:
        url = f"https://eodhistoricaldata.com/api/options/{ticker}.US?api_token={API_KEY}"

        response = requests.request("GET", url)

        OPTIONS[ticker] = response.json()
        OPTIONS[ticker]['expiries'] = [i['expirationDate'] for i in OPTIONS[ticker]['data']]
        OPTIONS[ticker]['options_chain'] = {}

        for sample in OPTIONS[ticker]['data']:
            opt_chain = {opt['strike']:{} for opt in sample['options']["CALL"]} if len(sample['options']["CALL"]) > len(sample['options']["PUT"]) else {opt['strike']:{} for opt in sample['options']["PUT"]}
            opt_types = ['CALL', 'PUT']
            for opt_type in opt_types:
                for opt in sample['options'][opt_type]: 
                    opt_chain[opt['strike']].update({RENAME_MAP[opt_type][k]:v for k, v in opt.items() if k in ['strike', 'bid', 'ask', 'volume', 'delta', 'gamma', 'theta', 'vega', 'rho', 'contractName']})
                    MEMORY[opt['contractName']] = opt
            
            OPTIONS[ticker]['options_chain'][sample['expirationDate']] = pd.DataFrame(list(opt_chain.values()), columns = ["Theta", "Gamma", "Rho", "Vega", "cName", "cDelta", "cVolume", "cBidP", "cTheo", "cAskP", "Strike", "pBidP", "pTheo", "pAskP", "pVolume", "pDelta", 'pName'])
    
    return OPTIONS[ticker]['data']

class OptionFromId:
    def __init__(self, contract_name, memory={}):
        self.contract_name = contract_name
        self.ticker = ""
        self.generate_ticker()

        query_data(self.ticker)

        if memory:
            self.memory = {contract_name: memory}
        else:
            self.memory = MEMORY
        
        self.stock_price = OPTIONS[self.ticker]['lastTradePrice']

        self.opt_type = self.memory[contract_name]['type']
        self.exercise_price =  self.memory[contract_name]['strike']
        self.time_to_expiration =  self.memory[contract_name]['daysBeforeExpiration']

        self.volatility = 0
        self.interest_rate = 0

    def generate_ticker(self):
        for i in self.contract_name:
            if i.isalpha():
                self.ticker += i
            else: break

class OptionByChoice:
    def __init__(self, ticker, opt_type, strike, expiration):
        self.data = query_data(ticker)
        self.strike = strike
        self.expiration = expiration

        opts_date = self.find_opt_date()
        self.opt_data = self.find_opt(opts_date['options'][opt_type])
        self.option = OptionFromId(self.opt_data['contractName'], memory=self.opt_data)

    def find_opt_date(self):
        for opt in self.data:
            if opt["expirationDate"] == self.expiration:
                return opt
        raise KeyError(f"No option with date: {self.expiration}")

    def find_opt(self, opts_datetype):
        for opt in opts_datetype:
            if opt['strike'] == self.strike:
                return opt
        raise KeyError(f"No option with date: {self.expiration}, strike: {self.strike}")

In [97]:
config = {
    'ticker': "AAPL"
}

# or 

config = {
    'ticker': "AAPL",
    'opt_type': 'CALL',
    'strike': 170,
    'expiration': '2022-02-25'
}

# or

config = {
    'id': "AAPL220218C00070000"
}

class OptionsPricer:
    def __init__(self, config):
        self.config = config
        if {'opt_type', 'strike', 'expiration'}.issubset(set(config.keys())):
            self.option = OptionByChoice(**config).option
        elif 'id' in config.keys():
            self.option = OptionFromId(config['id'])
        else:
            query_data(config['ticker'])
            print("See below expiries, and then `OPTIONS[option_chain][<expiry>]` to see options chains")
            print(OPTIONS['expiries'])

aapl = OptionsPricer(config)
aapl.option

In [99]:
MEMORY

{'AAPL220218C00070000': {'contractName': 'AAPL220218C00070000',
  'contractSize': 'REGULAR',
  'contractPeriod': 'MONTHLY',
  'currency': 'USD',
  'type': 'CALL',
  'inTheMoney': 'TRUE',
  'lastTradeDateTime': '2022-02-14 15:55:06',
  'expirationDate': '2022-02-18',
  'strike': 70,
  'lastPrice': 98.2,
  'bid': 102.3,
  'ask': 102.75,
  'change': -4.23,
  'changePercent': -0.0413,
  'volume': 1,
  'openInterest': 16,
  'impliedVolatility': 549.0118,
  'delta': 0.9923,
  'gamma': 0.0003,
  'theta': -0.3696,
  'vega': 0.0027,
  'rho': 0,
  'theoretical': 102.75,
  'intrinsicValue': 0,
  'timeValue': 0,
  'updatedAt': '2022-02-16 19:20:05',
  'daysBeforeExpiration': 1},
 'AAPL220218C00075000': {'contractName': 'AAPL220218C00075000',
  'contractSize': 'REGULAR',
  'contractPeriod': 'MONTHLY',
  'currency': 'USD',
  'type': 'CALL',
  'inTheMoney': 'TRUE',
  'lastTradeDateTime': '2022-02-09 14:51:22',
  'expirationDate': '2022-02-18',
  'strike': 75,
  'lastPrice': 100.85,
  'bid': 97.3,
  '

In [101]:
aapl.option.exercise_price

70

In [104]:
OPTIONS["AAPL"]["options_chain"]['2022-02-18']

,Theta,Gamma,Rho,Vega,cName,cDelta,cVolume,cBidP,cTheo,cAskP,Strike,pBidP,pTheo,pAskP,pVolume,pDelta,pName
0,0.0000,0.0000,0,0.0000,AAPL220218C00070000,0.9923,1.0,102.30,NaN,102.75,70.0,0.01,NaN,0.01,17.0,0.0000,AAPL220218P00070000
1,0.0000,0.0000,0,0.0000,AAPL220218C00075000,0.9917,12.0,97.30,NaN,97.75,75.0,0.01,NaN,0.01,722.0,0.0000,AAPL220218P00075000
2,0.0000,0.0000,0,0.0000,AAPL220218C00080000,0.9911,1.0,92.30,NaN,92.75,80.0,0.01,NaN,0.01,1.0,0.0000,AAPL220218P00080000
3,0.0000,0.0000,0,0.0000,AAPL220218C00085000,0.9904,3.0,87.25,NaN,87.75,85.0,0.01,NaN,0.01,1.0,0.0000,AAPL220218P00085000
4,0.0000,0.0000,0,0.0000,AAPL220218C00090000,0.9896,1.0,82.30,NaN,82.75,90.0,0.01,NaN,0.01,1.0,0.0000,AAPL220218P00090000
5,0.0000,0.0000,0,0.0000,AAPL220218C00095000,0.9888,5.0,77.30,NaN,77.75,95.0,0.01,NaN,0.01,45.0,0.0000,AAPL220218P00095000
6,0.0000,0.0000,0,0.0000,AAPL220218C00100000,0.9878,1.0,72.25,NaN,72.75,100.0,0.01,NaN,0.01,100.0,0.0000,AAPL220218P00100000
7,0.0000,0.0000,0,0.0000,AAPL220218C00105000,0.9868,23.0,67.30,NaN,67.75,105.0,0.01,NaN,0.01,1.0,0.0000,AAPL220218P00105000
8,0.0000,0.0000,0,0.0000,AAPL220218C00110000,0.9966,6.0,62.30,NaN,62.75,110.0,0.01,NaN,0.01,12.0,0.0000,AAPL220218P00110000
9,0.0000,0.0000,0,0.0000,AAPL220218C00115000,0.9844,2.0,57.30,NaN,57.75,115.0,0.01,NaN,0.01,2.0,0.0000,AAPL220218P00115000
